In [1]:
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import numpy as np
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from spacy.tokenizer import Tokenizer
from sklearn.neighbors import NearestNeighbors
import en_core_web_lg

In [2]:
df = pd.read_csv('cannabis.csv')
df

,Strain,Type,Rating,Effects,Flavor,Description
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...
2,1024,sativa,4.4,"Uplifted,Happy,Relaxed,Energetic,Creative","Spicy/Herbal,Sage,Woody",1024 is a sativa-dominant hybrid bred in Spain...
3,13-Dawgs,hybrid,4.2,"Tingly,Creative,Hungry,Relaxed,Uplifted","Apricot,Citrus,Grapefruit",13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,24K-Gold,hybrid,4.6,"Happy,Relaxed,Euphoric,Uplifted,Talkative","Citrus,Earthy,Orange","Also known as Kosher Tangie, 24k Gold is a 60%..."
...,...,...,...,...,...,...
2346,Zeus-Og,hybrid,4.7,"Happy,Uplifted,Relaxed,Euphoric,Energetic","Earthy,Woody,Pine",Zeus OG is a hybrid cross between Pineapple OG...
2347,Zkittlez,indica,4.6,"Relaxed,Happy,Euphoric,Uplifted,Sleepy","Sweet,Berry,Grape",Zkittlez is an indica-dominant mix of Grape Ap...
2348,Zombie-Kush,indica,5.0,"Relaxed,Sleepy,Talkative,Euphoric,Happy","Earthy,Sweet,Spicy/Herbal",Zombie Kush by Ripper Seeds comes from two dif...
2349,Zombie-Og,indica,4.4,"Relaxed,Sleepy,Euphoric,Happy,Hungry","Sweet,Earthy,Pungent",If you’re looking to transform into a flesh-ea...


In [3]:
# write a function to split each column into a list of list of words for EDA.

def make_into_list(words):
    lst = list(words.split(" "))
    return lst
        

test = make_into_list("Hello World")
test

['Hello', 'World']

In [4]:
df = df.replace('None', np.nan)
df = df.dropna().reset_index()

In [5]:
df['Flavor']= df['Flavor'].str.replace(',',' ')
df['Effects'] = df['Effects'].str.replace(',', ' ')

df.head()

,index,Strain,Type,Rating,Effects,Flavor,Description
0,0,100-Og,hybrid,4.0,Creative Energetic Tingly Euphoric Relaxed,Earthy Sweet Citrus,$100 OG is a 50/50 hybrid strain that packs a ...
1,1,98-White-Widow,hybrid,4.7,Relaxed Aroused Creative Happy Energetic,Flowery Violet Diesel,The ‘98 Aloha White Widow is an especially pot...
2,2,1024,sativa,4.4,Uplifted Happy Relaxed Energetic Creative,Spicy/Herbal Sage Woody,1024 is a sativa-dominant hybrid bred in Spain...
3,3,13-Dawgs,hybrid,4.2,Tingly Creative Hungry Relaxed Uplifted,Apricot Citrus Grapefruit,13 Dawgs is a hybrid of G13 and Chemdawg genet...
4,4,24K-Gold,hybrid,4.6,Happy Relaxed Euphoric Uplifted Talkative,Citrus Earthy Orange,"Also known as Kosher Tangie, 24k Gold is a 60%..."


In [6]:
df['combined'] = df["Effects"] +  df["Flavor"]

df

,index,Strain,Type,Rating,Effects,Flavor,Description,combined
0,0,100-Og,hybrid,4.0,Creative Energetic Tingly Euphoric Relaxed,Earthy Sweet Citrus,$100 OG is a 50/50 hybrid strain that packs a ...,Creative Energetic Tingly Euphoric RelaxedEart...
1,1,98-White-Widow,hybrid,4.7,Relaxed Aroused Creative Happy Energetic,Flowery Violet Diesel,The ‘98 Aloha White Widow is an especially pot...,Relaxed Aroused Creative Happy EnergeticFlower...
2,2,1024,sativa,4.4,Uplifted Happy Relaxed Energetic Creative,Spicy/Herbal Sage Woody,1024 is a sativa-dominant hybrid bred in Spain...,Uplifted Happy Relaxed Energetic CreativeSpicy...
3,3,13-Dawgs,hybrid,4.2,Tingly Creative Hungry Relaxed Uplifted,Apricot Citrus Grapefruit,13 Dawgs is a hybrid of G13 and Chemdawg genet...,Tingly Creative Hungry Relaxed UpliftedApricot...
4,4,24K-Gold,hybrid,4.6,Happy Relaxed Euphoric Uplifted Talkative,Citrus Earthy Orange,"Also known as Kosher Tangie, 24k Gold is a 60%...",Happy Relaxed Euphoric Uplifted TalkativeCitru...
...,...,...,...,...,...,...,...,...
2158,2346,Zeus-Og,hybrid,4.7,Happy Uplifted Relaxed Euphoric Energetic,Earthy Woody Pine,Zeus OG is a hybrid cross between Pineapple OG...,Happy Uplifted Relaxed Euphoric EnergeticEarth...
2159,2347,Zkittlez,indica,4.6,Relaxed Happy Euphoric Uplifted Sleepy,Sweet Berry Grape,Zkittlez is an indica-dominant mix of Grape Ap...,Relaxed Happy Euphoric Uplifted SleepySweet Be...
2160,2348,Zombie-Kush,indica,5.0,Relaxed Sleepy Talkative Euphoric Happy,Earthy Sweet Spicy/Herbal,Zombie Kush by Ripper Seeds comes from two dif...,Relaxed Sleepy Talkative Euphoric HappyEarthy ...
2161,2349,Zombie-Og,indica,4.4,Relaxed Sleepy Euphoric Happy Hungry,Sweet Earthy Pungent,If you’re looking to transform into a flesh-ea...,Relaxed Sleepy Euphoric Happy HungrySweet Eart...


In [7]:
print(spacy.__version__)  

2.3.2


In [8]:
nlp = en_core_web_lg.load()

In [9]:
# nlp = spacy.load("en_core_web_lg")
df.head()

# The Tokenizer
tokenizer = Tokenizer(nlp.vocab)

# Make the tokens for description
combined_tokens = []
for txt in tokenizer.pipe(df['combined'], batch_size=500):
    txt_tokens = [token.text for token in txt]
    combined_tokens.append(txt_tokens)
df['combined_tokens'] = combined_tokens
print(df['combined_tokens'].head())

0    [Creative, Energetic, Tingly, Euphoric, Relaxe...
1    [Relaxed, Aroused, Creative, Happy, EnergeticF...
2    [Uplifted, Happy, Relaxed, Energetic, Creative...
3    [Tingly, Creative, Hungry, Relaxed, UpliftedAp...
4    [Happy, Relaxed, Euphoric, Uplifted, Talkative...
Name: combined_tokens, dtype: object


In [10]:
df.head()

,index,Strain,Type,Rating,Effects,Flavor,Description,combined,combined_tokens
0,0,100-Og,hybrid,4.0,Creative Energetic Tingly Euphoric Relaxed,Earthy Sweet Citrus,$100 OG is a 50/50 hybrid strain that packs a ...,Creative Energetic Tingly Euphoric RelaxedEart...,"[Creative, Energetic, Tingly, Euphoric, Relaxe..."
1,1,98-White-Widow,hybrid,4.7,Relaxed Aroused Creative Happy Energetic,Flowery Violet Diesel,The ‘98 Aloha White Widow is an especially pot...,Relaxed Aroused Creative Happy EnergeticFlower...,"[Relaxed, Aroused, Creative, Happy, EnergeticF..."
2,2,1024,sativa,4.4,Uplifted Happy Relaxed Energetic Creative,Spicy/Herbal Sage Woody,1024 is a sativa-dominant hybrid bred in Spain...,Uplifted Happy Relaxed Energetic CreativeSpicy...,"[Uplifted, Happy, Relaxed, Energetic, Creative..."
3,3,13-Dawgs,hybrid,4.2,Tingly Creative Hungry Relaxed Uplifted,Apricot Citrus Grapefruit,13 Dawgs is a hybrid of G13 and Chemdawg genet...,Tingly Creative Hungry Relaxed UpliftedApricot...,"[Tingly, Creative, Hungry, Relaxed, UpliftedAp..."
4,4,24K-Gold,hybrid,4.6,Happy Relaxed Euphoric Uplifted Talkative,Citrus Earthy Orange,"Also known as Kosher Tangie, 24k Gold is a 60%...",Happy Relaxed Euphoric Uplifted TalkativeCitru...,"[Happy, Relaxed, Euphoric, Uplifted, Talkative..."


In [11]:
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [12]:
# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words = 'english',
                       ngram_range = (1,2),
                       max_features = 2000)

# Create a vocabulary and tf-idf score per document
dtm = tfidf.fit_transform(df['combined'])
                         

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
nn = NearestNeighbors(n_neighbors=4, algorithm='kd_tree')
nn.fit(dtm)


# View Feature Matrix as DataFrame
print(dtm.shape)
dtm.head()

(2163, 2000)


,ammonia,ammonia sweet,apple,apple blueberry,apple chestnut,apple earthy,apple pear,apricot,apricot earthy,apricot pine,...,woody flowery,woody nutty,woody pepper,woody pine,woody pungent,woody sage,woody skunk,woody spicy,woody sweet,woody tea
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
ideal = ["""
Creative,Uplifted,Tingly,Euphoric,Relaxed, Giggly
"""]

# Query the ideal descprition
new = tfidf.transform(ideal)
new

<1x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [14]:
nn.kneighbors(new.todense())

(array([[1.        , 1.10381499, 1.150652  , 1.15543082]]),
 array([[ 713,  373, 1912, 1881]]))

In [15]:
# most ideal description,strain and flavor

df['Description'][713]

'Ethiopian by Ace\xa0Seeds is a classic subtropical African sativa. This landrace exemplifies classic sativa morphology with its tall, bushy structure, vigorous growth, and natural mold resistance. Its long flowering time pays off with excellent final yields. Ethiopian’s effects will be appreciated by sativa lovers looking for an active, sociable high that is focused and stimulating. The bouquet is a melange of fermented citrus, wood chips, and musk. \xa0 \xa0'

In [16]:
df['Strain'][713]

'Ethiopian'

In [17]:
df['Flavor'][713]

'Coffee'

In [18]:
df['Effects'][713]

'Hungry'

In [23]:
df.head()

,index,Strain,Type,Rating,Effects,Flavor,Description,combined,combined_tokens
0,0,100-Og,hybrid,4.0,Creative Energetic Tingly Euphoric Relaxed,Earthy Sweet Citrus,$100 OG is a 50/50 hybrid strain that packs a ...,Creative Energetic Tingly Euphoric RelaxedEart...,"[Creative, Energetic, Tingly, Euphoric, Relaxe..."
1,1,98-White-Widow,hybrid,4.7,Relaxed Aroused Creative Happy Energetic,Flowery Violet Diesel,The ‘98 Aloha White Widow is an especially pot...,Relaxed Aroused Creative Happy EnergeticFlower...,"[Relaxed, Aroused, Creative, Happy, EnergeticF..."
2,2,1024,sativa,4.4,Uplifted Happy Relaxed Energetic Creative,Spicy/Herbal Sage Woody,1024 is a sativa-dominant hybrid bred in Spain...,Uplifted Happy Relaxed Energetic CreativeSpicy...,"[Uplifted, Happy, Relaxed, Energetic, Creative..."
3,3,13-Dawgs,hybrid,4.2,Tingly Creative Hungry Relaxed Uplifted,Apricot Citrus Grapefruit,13 Dawgs is a hybrid of G13 and Chemdawg genet...,Tingly Creative Hungry Relaxed UpliftedApricot...,"[Tingly, Creative, Hungry, Relaxed, UpliftedAp..."
4,4,24K-Gold,hybrid,4.6,Happy Relaxed Euphoric Uplifted Talkative,Citrus Earthy Orange,"Also known as Kosher Tangie, 24k Gold is a 60%...",Happy Relaxed Euphoric Uplifted TalkativeCitru...,"[Happy, Relaxed, Euphoric, Uplifted, Talkative..."


In [27]:
# find the row that corresponds to 713

